<a href="https://colab.research.google.com/github/VKSi/2020_12_DataCompetitions_Course/blob/main/HW_3_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Построение надежной локальной валидации


## Основное задание:
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые. Внимание! выборка assignment_2_test.csv - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!.

### Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

### Задание 1:
 сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

### Задание 2:
 сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

### Задание 3:
 построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

### Задание 4:
 выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

### Задание 5:
 сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

### Задание 6 (опциональное):
 сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.

### Задание 7 (совсем опциональное):
 в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).

# Общая (подготовительная) часть

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.3MB 54kB/s 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import files, drive
import pickle
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from typing import List, Tuple
import catboost as cb

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   33G   76G  30% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.9G     0  5.9G   0% /dev/shm
tmpfs           6.4G   16K  6.4G   1% /var/colab
/dev/sda1       114G   34G   81G  30% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  8.1G  7.0G  54% /content/gdrive


In [ ]:
!cp /content/gdrive/'My Drive'/AI/GB/Kaggle/'assignment_2_test.csv' .
!cp /content/gdrive/'My Drive'/AI/GB/Kaggle/'assignment_2_train.csv' .

In [ ]:
object_path = "gdrive/My Drive/AI/GB/Kaggle/objects/"

In [ ]:
def save_obj(obj, name: str):
    path = object_path + name + '.pkl'
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    return path


def load_obj(name: str):
    with open(object_path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
data = pd.read_csv("assignment_2_train.csv")
lb = pd.read_csv("assignment_2_test.csv")

In [ ]:
print("data.shape = {} rows, {} cols".format(*data.shape))
print("lb.shape = {} rows, {} cols".format(*lb.shape))

data.shape = 180000 rows, 394 cols
lb.shape = 100001 rows, 394 cols


In [ ]:
REST_FEATURES = data.columns
INITIAL_FEATURES = REST_FEATURES.copy()

ID = 'TransactionID'
REST_FEATURES = list(set(REST_FEATURES) - set([ID]))

TARGET = 'isFraud'
REST_FEATURES = list(set(REST_FEATURES) - set([TARGET]))

NUMERICAL_FEATURES = data[REST_FEATURES].select_dtypes(include=[np.number]).columns.to_list()
REST_FEATURES = list(set(REST_FEATURES) - set(NUMERICAL_FEATURES))

CATEGORICAL_FEATURES = REST_FEATURES

REST_FEATURES = None

In [ ]:
data[TARGET].value_counts()/data.shape[0]

0    0.971439
1    0.028561
Name: isFraud, dtype: float64

In [ ]:
data[TARGET].value_counts()/data.shape[0]

0    0.971439
1    0.028561
Name: isFraud, dtype: float64

In [ ]:
data[CATEGORICAL_FEATURES] = data[CATEGORICAL_FEATURES].astype('str')
lb[CATEGORICAL_FEATURES] = lb[CATEGORICAL_FEATURES].astype('str')

In [ ]:
X_lb = lb.drop([ID, TARGET], axis='columns')
y_lb = lb[TARGET]

In [ ]:
cb_params = {
    "n_estimators": 10000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 100,
    "max_depth": 4,
    "reg_lambda": 100,
    "class_weights":[3, 97], 
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 27
}

# Задание 1:
сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [ ]:
data[ID].nunique() == data.shape[0]

True

Все значения ID уникальны

В задании требуется разбить данные по ID, для этого их нужно упорядочить

In [ ]:
data.sort_values(by=ID, inplace=True)

In [ ]:
X = data.drop([ID, TARGET], axis='columns')
y = data[TARGET]

In [ ]:
print(f'X.shape = {X.shape}')
print(f'y.shape = {y.shape}')

X.shape = (180000, 392)
y.shape = (180000,)


In [ ]:
train_share = 0.25

In [ ]:
X_train = X.head(int(X.shape[0]*(1 - train_share)))
y_train = y.loc[y.index.isin(X_train.index)]
X_test = X.loc[~X.index.isin(X_train.index)]
y_test = y.loc[y.index.isin(X_test.index)]

In [ ]:
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}')

X_train.shape = (135000, 392)
y_train.shape = (135000,)
X_test.shape = (45000, 392)
y_test.shape = (45000,)


In [ ]:
train_pool = cb.Pool(X_train, y_train, cat_features=CATEGORICAL_FEATURES)
valid_pool = cb.Pool(X_test, y_test, cat_features=CATEGORICAL_FEATURES)

In [ ]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(train_pool, eval_set=[train_pool, valid_pool])

0:	test: 0.7514447	test1: 0.7402343	best: 0.7402343 (0)	total: 759ms	remaining: 37m 57s
100:	test: 0.8740239	test1: 0.8495840	best: 0.8495840 (100)	total: 1m	remaining: 28m 49s
200:	test: 0.8851725	test1: 0.8576180	best: 0.8576180 (200)	total: 1m 59s	remaining: 27m 41s
300:	test: 0.8921213	test1: 0.8639272	best: 0.8639272 (300)	total: 2m 57s	remaining: 26m 27s
400:	test: 0.8981703	test1: 0.8679228	best: 0.8679228 (400)	total: 3m 54s	remaining: 25m 20s
500:	test: 0.9103926	test1: 0.8771553	best: 0.8771553 (500)	total: 4m 53s	remaining: 24m 25s
600:	test: 0.9191184	test1: 0.8827751	best: 0.8827751 (600)	total: 5m 51s	remaining: 23m 24s
700:	test: 0.9252989	test1: 0.8859906	best: 0.8859906 (700)	total: 6m 50s	remaining: 22m 26s
800:	test: 0.9304810	test1: 0.8891957	best: 0.8892175 (797)	total: 7m 52s	remaining: 21m 37s
900:	test: 0.9345428	test1: 0.8911036	best: 0.8911996 (898)	total: 8m 50s	remaining: 20m 35s
1000:	test: 0.9383434	test1: 0.8929068	best: 0.8929260 (988)	total: 9m 48s	rema

In [ ]:
save_obj(model, 'model_task_1')

'gdrive/My Drive/AI/GB/Kaggle/objects/model_task_1.pkl'

In [ ]:
train_score = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
test_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
lb_score = roc_auc_score(y_lb, model.predict_proba(X_lb)[:, 1])

print(f"Train-score: {round(train_score, 3)}, Test-score: {round(test_score, 3)} LB-score: {round(lb_score, 3)}")

Train-score: 0.964, Test-score: 0.906 LB-score: 0.861


#### Вывод:
модель выглядит переученной: метрика на тесте ниже, чем метрика на трейне. При этом метрика на ЛиберБорд еще ниже.

# Задание 2:
 сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [ ]:
train_share = 0.3

In [ ]:
X_train = X.head(int(X.shape[0]*(1 - train_share)))
y_train = y.loc[y.index.isin(X_train.index)]
X_test = X.loc[~X.index.isin(X_train.index)]
y_test = y.loc[y.index.isin(X_test.index)]

X_valid = X_test.head(int(X_test.shape[0]*(1 - train_share)))
y_valid = y_test.loc[y_test.index.isin(X_valid.index)]
X_test = X_test.loc[~X_test.index.isin(X_valid.index)]
y_test = y_test.loc[y_test.index.isin(X_test.index)]

In [ ]:
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'X_valid.shape = {X_valid.shape}')
print(f'y_valid.shape = {y_valid.shape}')
print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}')

X_train.shape = (125999, 392)
y_train.shape = (125999,)
X_valid.shape = (37800, 392)
y_valid.shape = (37800,)
X_test.shape = (16201, 392)
y_test.shape = (16201,)


In [ ]:
train_pool = cb.Pool(X_train, y_train, cat_features=CATEGORICAL_FEATURES)
valid_pool = cb.Pool(X_valid, y_valid, cat_features=CATEGORICAL_FEATURES)

In [ ]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(train_pool, eval_set=[train_pool, valid_pool])

0:	test: 0.7588777	test1: 0.7886359	best: 0.7886359 (0)	total: 719ms	remaining: 35m 55s
100:	test: 0.8749803	test1: 0.8611416	best: 0.8611692 (98)	total: 56.6s	remaining: 27m 5s
200:	test: 0.8864823	test1: 0.8689309	best: 0.8689516 (199)	total: 1m 53s	remaining: 26m 20s
300:	test: 0.8936365	test1: 0.8730562	best: 0.8730579 (297)	total: 2m 46s	remaining: 24m 56s
400:	test: 0.9006290	test1: 0.8778237	best: 0.8778237 (400)	total: 3m 40s	remaining: 23m 49s
500:	test: 0.9121103	test1: 0.8856522	best: 0.8856522 (500)	total: 4m 35s	remaining: 22m 54s
600:	test: 0.9201918	test1: 0.8903625	best: 0.8903625 (600)	total: 5m 29s	remaining: 21m 54s
700:	test: 0.9257604	test1: 0.8945793	best: 0.8945793 (700)	total: 6m 22s	remaining: 20m 54s
800:	test: 0.9306055	test1: 0.8970273	best: 0.8970273 (800)	total: 7m 17s	remaining: 20m
900:	test: 0.9347594	test1: 0.8990032	best: 0.8990070 (899)	total: 8m 11s	remaining: 19m 6s
1000:	test: 0.9389401	test1: 0.9013326	best: 0.9013326 (1000)	total: 9m 7s	remainin

In [ ]:
save_obj(model, 'model_task_2')

'gdrive/My Drive/AI/GB/Kaggle/objects/model_task_2.pkl'

In [ ]:
train_score = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
test_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
valid_score = roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])
lb_score = roc_auc_score(y_lb, model.predict_proba(X_lb)[:, 1])

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)} LB-score: {round(lb_score, 3)}")

Train-score: 0.952, Valid-score: 0.905, Test-score: 0.88 LB-score: 0.866


"Динамика" от трейна к тесту через валидационную выборку дает более точное представление о том, что ждать от Либерборда

# Задание 3:
построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

In [ ]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [ ]:
np.random.seed(26)
scores = create_bootstrap_metrics(y_test, model.predict_proba(X_test)[:, 1], roc_auc_score)

calculate_confidence_interval(scores)

(0.8646961283373846, 0.8950432004637787)

Ура! Доверительный интервал достаточно узкий. Результат на Лидерборде лежит внутри доверительного интервала.     

Реально, мы должны были использовать валидационный набор для подбора гиперпараметров и только потом лучшую модель оценивать на тесте. Но в задании этого не требуется.

# Задание 4:
выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

In [ ]:
X_adv = pd.concat([X, X_lb])
y_adv = np.hstack((np.ones(X_lb.shape[0]), np.zeros(X.shape[0])))

In [ ]:
print(f'X_adv.shape = {X_adv.shape}')
print(f'y_adv.shape = {y_adv.shape}')

X_adv.shape = (280001, 392)
y_adv.shape = (280001,)


In [ ]:
train_pool = cb.Pool(X_adv, y_adv, cat_features=CATEGORICAL_FEATURES)

In [ ]:
weight_1 = (y_adv == 1).sum()
weight_0 = (y_adv == 0).sum()
print(weight_1, weight_0)

100001 180000


In [ ]:
cb_params_adv = cb_params.copy()
cb_params_adv['class_weights'] = [weight_1, weight_0]

In [ ]:
model = cb.CatBoostClassifier(**cb_params_adv)
model.fit(train_pool, eval_set=[train_pool])

0:	test: 0.5691879	best: 0.5691879 (0)	total: 1.23s	remaining: 1h 1m 19s
100:	test: 0.9978272	best: 0.9979014 (90)	total: 1m 40s	remaining: 48m 12s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9981597609
bestIteration = 108

Shrink model to first 109 iterations.


In [ ]:
save_obj(model, 'model_task_4_adv')

'gdrive/My Drive/AI/GB/Kaggle/objects/model_task_4_adv.pkl'

In [ ]:
proba = model.predict_proba(X)

In [ ]:
(proba[:, 1] > 0.5).sum()/proba.shape[0]

0.5844333333333334

Подберем уровень отсечки чтобы отобрать 25% "самых" похожих на лидерборд

In [ ]:
th = 0.82
(proba[:, 1] > th).sum()/proba.shape[0]

0.25875555555555557

In [ ]:
to_train_ind = np.where(proba[:, 1] <= th)[0]
to_test_ind = np.where(proba[:, 1] > th)[0]

In [ ]:
X_train = X.loc[X.index.isin(to_train_ind)]
y_train = y.loc[y.index.isin(X_train.index)]
X_test = X.loc[X.index.isin(to_test_ind)]
y_test = y.loc[y.index.isin(X_test.index)]

In [ ]:
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}')

X_train.shape = (133424, 392)
y_train.shape = (133424,)
X_test.shape = (46576, 392)
y_test.shape = (46576,)


In [ ]:
train_pool = cb.Pool(X_train, y_train, cat_features=CATEGORICAL_FEATURES)
valid_pool = cb.Pool(X_valid, y_valid, cat_features=CATEGORICAL_FEATURES)

In [ ]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(train_pool, eval_set=[train_pool, valid_pool])

0:	test: 0.7655423	test1: 0.7876907	best: 0.7876907 (0)	total: 760ms	remaining: 2h 6m 40s
100:	test: 0.8665136	test1: 0.8809609	best: 0.8809609 (100)	total: 59s	remaining: 1h 36m 19s
200:	test: 0.8827677	test1: 0.8944929	best: 0.8944929 (200)	total: 1m 55s	remaining: 1h 33m 56s
300:	test: 0.8909067	test1: 0.9011871	best: 0.9011871 (300)	total: 2m 50s	remaining: 1h 31m 47s
400:	test: 0.8986673	test1: 0.9074538	best: 0.9074538 (400)	total: 3m 46s	remaining: 1h 30m 30s
500:	test: 0.9096786	test1: 0.9167436	best: 0.9167436 (500)	total: 4m 44s	remaining: 1h 29m 59s
600:	test: 0.9187632	test1: 0.9243692	best: 0.9243692 (600)	total: 5m 42s	remaining: 1h 29m 22s
700:	test: 0.9257627	test1: 0.9303281	best: 0.9303281 (700)	total: 6m 42s	remaining: 1h 29m 2s
800:	test: 0.9316076	test1: 0.9351679	best: 0.9351679 (800)	total: 7m 41s	remaining: 1h 28m 25s
900:	test: 0.9361129	test1: 0.9392100	best: 0.9392100 (900)	total: 8m 39s	remaining: 1h 27m 29s
1000:	test: 0.9400540	test1: 0.9428313	best: 0.942

In [ ]:
save_obj(model, 'model_task_4')

'gdrive/My Drive/AI/GB/Kaggle/objects/model_task_4.pkl'

In [ ]:
train_score = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
test_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
lb_score = roc_auc_score(y_lb, model.predict_proba(X_lb)[:, 1])

print(f"Train-score: {round(train_score, 3)}, Test-score: {round(test_score, 3)} LB-score: {round(lb_score, 3)}")

Train-score: 0.989, Test-score: 0.919 LB-score: 0.838


### Вывод
Модель училась гораздо дольше и все-равно остановилась не по критерию останова. При этом метрика качества на обучающей и валидационной выборках существенно выше, чем в предыдущих вариантах.     
Удивительно, но в процессе обучения результаты на тесте были не только гораздо ближе к трейну, чем в предыдущих вариантах, но практически постоянно превышали их.
К сожалению, результат на ЛидерБорде ниже (!) чем в первом задании.    


# Задание 5:
 сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

Доработаем функцию из лекции, чтобы она работала с catboost

In [ ]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        train_pool = cb.Pool(X_train, y_train, cat_features=CATEGORICAL_FEATURES)
        valid_pool = cb.Pool(X_valid, y_valid, cat_features=CATEGORICAL_FEATURES)
        estimator.fit(train_pool, eval_set=[train_pool, valid_pool])

        y_train_pred = estimator.predict(X_train)
        y_valid_pred = estimator.predict(X_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)
        save_obj(estimators, 'estimators_task_5')

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [ ]:
cb_params_3 = cb_params.copy()
cb_params_3['silent'] = True
cb_params_3['verbose'] = None
model = cb.CatBoostClassifier(**cb_params_3)

In [ ]:
cv_strategy = StratifiedKFold(n_splits=5)

In [ ]:
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = \
    make_cross_validation(X, y, model, metric=roc_auc_score, cv_strategy=cv_strategy)

Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.8863, valid-score = 0.8078
Fold: 2, train-observations = 144000, valid-observations = 36000
train-score = 0.8922, valid-score = 0.8306
Fold: 3, train-observations = 144000, valid-observations = 36000
train-score = 0.7874, valid-score = 0.7741
Fold: 4, train-observations = 144000, valid-observations = 36000
train-score = 0.7838, valid-score = 0.7891
Fold: 5, train-observations = 144000, valid-observations = 36000
train-score = 0.9003, valid-score = 0.7966
CV-results train: 0.85 +/- 0.053
CV-results valid: 0.7996 +/- 0.019
OOF-score = 0.7996


In [ ]:
save_obj(estimators, 'estimators_task_5')

'gdrive/My Drive/AI/GB/Kaggle/objects/estimators_task_5.pkl'

In [ ]:
proba = np.zeros((len(estimators), X_lb.shape[0]))
for i, est in enumerate(estimators):
  proba[i, :] = est.predict_proba(X_lb)[:, 1]
proba_lb = np.mean(proba, axis = 0)
proba_lb
 

array([0.93457839, 0.24743438, 0.40196993, ..., 0.60295161, 0.8435361 ,
       0.24872083])

In [ ]:
y_pred = (proba_lb > 0.5) 

In [ ]:
lb_score = roc_auc_score(y_lb, y_pred)
print(f'LB score = {round(lb_score, 3)}')

LB score = 0.768


Результаты на валидационном наборе данных (минус одна сигма от среднего) находятся за пределами 1-й сигмы тренировочного набора.      
Результат на Лидерборде находится за пределами того и другого.